In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

## Import and clean data

In [35]:
# Import and clean variable metadata
metadata = pd.read_csv('qol-data/csvFiles/metadata.csv')
metadata = metadata.drop(0)
metadata = metadata.iloc[:, 0:5]
metadata = metadata.dropna()


In [36]:
# Get selected NPAs
selected = pd.read_csv("./selectedNPA.csv")
selected['order'] = selected.index

In [37]:
characterRaw = pd.read_csv('qol-data/csvFiles/character.csv', header=1).dropna(thresh=10)
characterRaw = characterRaw[characterRaw.NPA.notnull()]
characterRaw.NPA = characterRaw.NPA.astype(int)

characterMerged = pd.merge(selected, characterRaw, left_on="NPA",right_on="NPA",how="left")
characterMerged.to_pickle('./qol-data/pickles/character.pkl')

In [38]:
economyRaw = pd.read_csv('qol-data/csvFiles/economy.csv', header=1).dropna(thresh=10)
economyRaw = economyRaw[economyRaw.NPA.notnull()]
economyRaw.NPA = economyRaw.NPA.astype(int)

economyMerged = pd.merge(selected,economyRaw,left_on="NPA",right_on="NPA",how="left")
economyMerged[['Household_Income_2018','Household_Income_moe_2018']] = economyMerged[['Household_Income_2018','Household_Income_moe_2018']].astype(float)
economyMerged.to_pickle('./qol-data/pickles/economy.pkl')

In [39]:
educationRaw = pd.read_csv('qol-data/csvFiles/education.csv',header=1).dropna(thresh=10)
educationRaw = educationRaw[educationRaw.NPA.notnull()]
educationRaw.NPA = educationRaw.NPA.astype(int)

educationMerged = pd.merge(selected,educationRaw,left_on="NPA",right_on="NPA",how="left")
educationMerged[['Bachelors_Degree_2018','Bachelors_Degree_moe_2018','High_School_Diploma_2018','High_School_Diploma_moe_2018']] = educationMerged[['Bachelors_Degree_2018','Bachelors_Degree_moe_2018','High_School_Diploma_2018','High_School_Diploma_moe_2018']].astype(float)
educationMerged.to_pickle('./qol-data/pickles/education.pkl')

In [40]:
engagementRaw = pd.read_csv('qol-data/csvFiles/engagement.csv',header=1).dropna(thresh=10)
engagementRaw = engagementRaw[engagementRaw.NPA.notnull()]
engagementRaw.NPA = engagementRaw.NPA.astype(int)

engagementMerged = pd.merge(selected,engagementRaw,left_on="NPA",right_on="NPA",how="left")
engagementMerged.to_pickle('./qol-data/pickles/engagement.pkl')

In [41]:
environmentRaw = pd.read_csv('qol-data/csvFiles/environment.csv',header=1).dropna(thresh=10)
environmentRaw = environmentRaw[environmentRaw.NPA.notnull()]
environmentRaw.NPA = environmentRaw.NPA.astype(int)

environmentMerged = pd.merge(selected,environmentRaw,left_on="NPA",right_on="NPA",how="left")
environmentMerged.to_pickle('./qol-data/pickles/environment.pkl')

In [42]:
healthRaw = pd.read_csv('qol-data/csvFiles/health.csv',header=1).dropna(thresh=10)
healthRaw = healthRaw[healthRaw.NPA.notnull()]
healthRaw.NPA = healthRaw.NPA.astype(int)

healthMerged = pd.merge(selected,healthRaw,left_on="NPA",right_on="NPA",how="left")
healthMerged.to_pickle('./qol-data/pickles/health.pkl')

In [43]:
housingRaw = pd.read_csv('qol-data/csvFiles/housing.csv',header=1).dropna(thresh=10)
housingRaw = housingRaw[housingRaw.NPA.notnull()]
housingRaw.NPA = housingRaw.NPA.astype(int)

housingMerged = pd.merge(selected,housingRaw,left_on="NPA",right_on="NPA",how="left")
housingMerged[list(housingMerged.select_dtypes(include=['object']).columns)] = housingMerged[list(housingMerged.select_dtypes(include=['object']).columns)].astype(float)
housingMerged.to_pickle('./qol-data/pickles/housing.pkl')

In [44]:
safetyRaw = pd.read_csv('qol-data/csvFiles/safety.csv',header=1).dropna(thresh=10)
safetyRaw = safetyRaw[safetyRaw.NPA.notnull()]
safetyRaw.NPA = safetyRaw.NPA.astype(int)

safetyMerged = pd.merge(selected,safetyRaw,left_on="NPA",right_on="NPA",how="left")
safetyMerged['Property_Crime_Rate_2011'] = safetyMerged['Property_Crime_Rate_2011'].astype(float)
safetyMerged.to_pickle('./qol-data/pickles/safety.pkl')

In [45]:
transportationRaw = pd.read_csv('qol-data/csvFiles/transportation.csv',header=1).dropna(thresh=10)
transportationRaw = transportationRaw[transportationRaw.NPA.notnull()]
transportationRaw.NPA = transportationRaw.NPA.astype(int)

transportationMerged = pd.merge(selected,transportationRaw,left_on="NPA",right_on="NPA",how="left")
transportationMerged[['Long_Commute_2018','Long_Commute_moe_2018']] = transportationMerged[['Long_Commute_2018','Long_Commute_moe_2018']].astype(float)
transportationMerged.to_pickle('./qol-data/pickles/transportation.pkl')

## Combine filtered data into one file

In [46]:
import os
from functools import reduce
dataFiles = os.listdir('./qol-data/pickles/')
dataFiles

['character.pkl',
 'economy.pkl',
 'education.pkl',
 'engagement.pkl',
 'environment.pkl',
 'health.pkl',
 'housing.pkl',
 'safety.pkl',
 'transportation.pkl']

In [47]:
def getPickle(file):
    return pd.read_pickle('./qol-data/pickles/{}'.format(file))

# List comprehension to create list of dataframes
dataFrames = [getPickle(file) for file in dataFiles]


In [48]:
# Reduce this list to one master frame with left joins
master = reduce(lambda left,right: pd.merge(left,right,on=["NPA","order"],how="left"),dataFrames)

In [49]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Columns: 585 entries, NPA to Transit_Ridership_Total_2013
dtypes: float64(563), int64(2), object(20)
memory usage: 77.8+ KB


In [50]:
master.select_dtypes('object')

,Youth_Population_2018,Youth_Population_moe_2018,Older_Adult_Population_2018,Older_Adult_Population_moe_2018,White_Population_2018,White_Population_moe_2018,Black_Population_2018,Black_Population_moe_2018,Black_Population_moe_2017,Asian_Population_2018,Asian_Population_moe_2018,All_Other_Races_2018,All_Other_Races_moe_2018,Hispanic_Latino_2018,Hispanic_Latino_moe_2018,Employment_Rate_2018,Employment_Rate_moe_2018,Impervious_Surface_2011,Commuters_Driving_Alone_2018,Commuters_Driving_Alone_moe_2018
0,29,4,8,4,76,8,9,7,5,8,2,0,2,6,4,94,22,8.5,94,101
1,33,8,5,3,74,6,9,9,8,1,2,0,2,6,10,#DIV/0!,#DIV/0!,4.4,90,114
2,25,5,12,3,79,8,11,6,6,3,2,0,2,6,3,92,33,15,91,97
3,30,6,6,5,71,10,17,7,6,0,2,0,2,4,3,96,14,10.7,86,101
4,24,5,11,3,69,4,16,7,11,1,2,0,2,9,7,97,43,9.5,87,93
5,28,5,7,2,37,6,45,5,7,4,2,0,2,13,6,83,25,15.6,89,93
6,22,6,5,2,41,11,32,12,10,3,2,0,2,21,13,100,173,15.3,86,107
7,21,6,7,3,25,8,52,12,11,1,2,0,2,12,8,91,29,20.3,95,96
8,30,7,8,5,17,9,68,17,14,0,2,0,2,8,6,95,25,5.6,85,99
9,16,6,12,3,12,3,81,4,6,0,2,0,2,6,3,91,30,13.2,91,102


In [51]:
master.loc[1,['Employment_Rate_2018','Employment_Rate_moe_2018']] = ['NaN','NaN']
objList = list(master.select_dtypes('object').columns)
master[objList] = master[objList].astype(float)

In [52]:
master.to_pickle('./qol-data/master.pkl')